In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
from scipy.signal import butter, filtfilt, windows
import matplotlib.cm as cm

save_folder = 'D:/THESE/Git_Scripts/Python_Scripts/motoneuron_simulation/'

duration = 80 # in seconds
fsamp = 1000 # in samples/s
nb_signals_to_generate = 10 # > 1 necessary for scripts requiring several independent inputs
low_pass_cutoff = 5 # in hz

# Low-pass filter artifact removal
duration_to_remove = 1/low_pass_cutoff # in second
Wind_s = duration_to_remove * 2
HanningW = windows.hann(round(fsamp * Wind_s))
HanningW = HanningW[:int(np.round(len(HanningW)/2))]
nb_samples_artifact_removal_window = len(HanningW)

time = np.linspace(0,duration,int(duration*fsamp))
plt.figure()
plt.plot(HanningW)
plt.plot(HanningW[::-1])

In [ ]:
# Define a low-pass filter
def butter_lowpass(cutoff, fs, order=5):
    nyquist = 0.5 * fs
    normal_cutoff = cutoff / nyquist
    b, a = butter(order, normal_cutoff, btype='low', analog=False)
    return b, a
def lowpass_filter(data, cutoff, fs, order=5):
    b, a = butter_lowpass(cutoff, fs, order=order)
    y = filtfilt(b, a, data)
    return y

In [ ]:
colormap_temp = cm.get_cmap('plasma')
signals = []
plt.figure(figsize=(25,7))
for signali in range(nb_signals_to_generate):
    temp_sig = np.random.normal(0, 1, len(time))
    temp_sig = lowpass_filter(temp_sig, low_pass_cutoff, fsamp)
    # Remove artifacts of the low-pass filter at the edges
    # start of signal - artifact removal
    temp_sig[0:nb_samples_artifact_removal_window] = temp_sig[0:nb_samples_artifact_removal_window] * HanningW
     # end of signal - artifact removal
    temp_sig[len(temp_sig)-nb_samples_artifact_removal_window:len(temp_sig)] = temp_sig[len(temp_sig)-nb_samples_artifact_removal_window:len(temp_sig)] * HanningW[::-1]
    # Normalize the signal (mean-centered; std = 1)
    temp_sig = temp_sig - np.mean(temp_sig)
    temp_sig = temp_sig / np.std(temp_sig)
    signals.append(temp_sig)
    plt.plot(temp_sig,label=f'signal #{signali+1}',alpha=np.sqrt(1/nb_signals_to_generate), color=colormap_temp(signali/nb_signals_to_generate) )
plt.legend()
plt.xlabel("time (samples)")
plt.ylabel("Amplitude (normalized)")
# Convert into dataframe to save as a csv (one column per signal, one row per sample)
df = pd.DataFrame(signals)
df = df.transpose()

In [ ]:
new_filename = f'Synthetic_signals.csv'
save_file_path = os.path.join(save_folder, new_filename)
df.to_csv(save_file_path, index=False)